<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Pre-Process-Files" data-toc-modified-id="Pre-Process-Files-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Pre-Process Files</a></span></li><li><span><a href="#Create-Tables-and-Load-CSV-Data" data-toc-modified-id="Create-Tables-and-Load-CSV-Data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Create Tables and Load CSV Data</a></span><ul class="toc-item"><li><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Create-Cluster" data-toc-modified-id="Create-Cluster-3.0.0.1"><span class="toc-item-num">3.0.0.1&nbsp;&nbsp;</span>Create Cluster</a></span></li><li><span><a href="#Create-Keyspace" data-toc-modified-id="Create-Keyspace-3.0.0.2"><span class="toc-item-num">3.0.0.2&nbsp;&nbsp;</span>Create Keyspace</a></span></li><li><span><a href="#Set-Keyspace" data-toc-modified-id="Set-Keyspace-3.0.0.3"><span class="toc-item-num">3.0.0.3&nbsp;&nbsp;</span>Set Keyspace</a></span></li></ul></li></ul></li></ul></li><li><span><a href="#Create-Tables" data-toc-modified-id="Create-Tables-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Create Tables</a></span><ul class="toc-item"><li><span><a href="#Table-1:-song" data-toc-modified-id="Table-1:-song-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Table 1: song</a></span><ul class="toc-item"><li><span><a href="#Composite-Primary-Key" data-toc-modified-id="Composite-Primary-Key-4.1.1"><span class="toc-item-num">4.1.1&nbsp;&nbsp;</span>Composite Primary Key</a></span></li><li><span><a href="#Validate-table-and-SELECT-query" data-toc-modified-id="Validate-table-and-SELECT-query-4.1.2"><span class="toc-item-num">4.1.2&nbsp;&nbsp;</span>Validate table and SELECT query</a></span></li></ul></li><li><span><a href="#Table-2:-user_playlist_session" data-toc-modified-id="Table-2:-user_playlist_session-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Table 2: user_playlist_session</a></span></li><li><span><a href="#Composite-Primary-Key" data-toc-modified-id="Composite-Primary-Key-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Composite Primary Key</a></span><ul class="toc-item"><li><span><a href="#Validate-table-and-SELECT-query" data-toc-modified-id="Validate-table-and-SELECT-query-4.3.1"><span class="toc-item-num">4.3.1&nbsp;&nbsp;</span>Validate table and SELECT query</a></span></li></ul></li><li><span><a href="#Table-3:-username" data-toc-modified-id="Table-3:-username-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Table 3: username</a></span><ul class="toc-item"><li><span><a href="#Example-Request" data-toc-modified-id="Example-Request-4.4.1"><span class="toc-item-num">4.4.1&nbsp;&nbsp;</span>Example Request</a></span></li><li><span><a href="#Primary-Key" data-toc-modified-id="Primary-Key-4.4.2"><span class="toc-item-num">4.4.2&nbsp;&nbsp;</span>Primary Key</a></span></li><li><span><a href="#Validate-table-and-select-query" data-toc-modified-id="Validate-table-and-select-query-4.4.3"><span class="toc-item-num">4.4.3&nbsp;&nbsp;</span>Validate table and select query</a></span></li></ul></li></ul></li><li><span><a href="#Drop-Tables" data-toc-modified-id="Drop-Tables-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Drop Tables</a></span></li><li><span><a href="#Close-Session-and-Cluster-Connection¶" data-toc-modified-id="Close-Session-and-Cluster-Connection¶-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Close Session and Cluster Connection¶</a></span></li></ul></div>

__<font size="6">Apache Cassandra ETL Notebook</font>__

# Imports

In [1]:
import cassandra
from cassandra.cluster import Cluster
from cassandra.query import ordered_dict_factory

import pandas as pd
import re
import os
import glob
import numpy as np
import json
import csv

# Pre-Process Files

Creating list of filepaths to process original event csv data files

In [2]:
# checking current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):  
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list, read csv file and load data to full_data_rows_list
for f in file_path_list:

    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Create Tables and Load CSV Data

The image below is a screenshot of what the denormalized data should appear like in the event_datafile_new.csv after preprocssing code above is executed:<br>

<img src="images/image_event_datafile_new.jpg">

#### Create Cluster

In [2]:
# init cluster on local machine
cluster = Cluster(contact_points=['127.0.0.1'])

# create session to establish connection and begin executing queries
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 

try:
    session.execute("""
    CREATE keyspace 
    IF NOT EXISTS sparkify WITH replication = {'class' : 'SimpleStrategy', 
                                               'replication_factor' : 1}
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace("sparkify")
except Exception as e:
    print(e)

# Create Tables

## Table 1: song

Example Request: Get the `artist`, `song` title and song's `length` in the music app history that was heard during `sessionId = 338`, and `itemInSession = 4`

### Composite Primary Key
The `sessionid` column is the partition key because the associated `SELECT` query will filter results by `sessionId`. Having the partition key reflect the `SELECT` query will decrease query latency. `itemInSession` is the clustering column to prevent data integrity issues regarding overidden duplicate values.

In [8]:
# example query that the model will be based on
song_query = """
SELECT 
    artist, 
    song, 
    length 
FROM song 
WHERE sessionid = 338 
    AND itemInSession = 4 
"""

In [9]:
#create song table 
song_table = """ 
CREATE TABLE IF NOT EXISTS song 
    (
        sessionid       INT, 
        itemInSession   INT, 
        artist          TEXT, 
        length          FLOAT, 
        song            TEXT,
        PRIMARY KEY     (sessionid, itemInSession) 
    )
"""

try:
    session.execute(song_table)
except Exception as e:
    print(e)

In [10]:
#extract csv data
file = 'event_datafile_new.csv'

# extract apporiate data from csv and load into song table
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song (sessionId, itemInSession, artist, length, song)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        # rearrange order of csv values to match insert query 
        session.execute(query, ([int(line[8]), int(line[3]), line[0], float(line[5]), line[9]]))

### Validate table and SELECT query

In [11]:
#output of SELECT queries will be ordered dictionaries (used for creating output dataframe)
session.row_factory = ordered_dict_factory

#execute SELECT query
try:
    rows = session.execute(song_query)
except Exception as e:
    print(e)

#initialize output dataframe
song_df = pd.DataFrame(columns=['artist', 'length', 'song'])
#append query output to dataframe
for row in rows:
    song_df = song_df.append(row, ignore_index=True)
    
song_df.head()

,artist,length,song
0,Faithless,495.307312,Music Matters (Mark Knight Dub)


## Table 2: user_playlist_session

Example Request: Get name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

## Composite Primary Key
`userId` and `sessionId` are the partition keys because the associated `SELECT` query will filter results by `userId` and `sessionId`. Having the partition key reflect the `SELECT` query will decrease query latency. The `sessionId` column is the second partition key to prevent data integrity issues and to keep sessions from the same `userId` together within the same node.  `itemInSession` is the clustering column because the query requests the output to be ordered by `itemInSession`.

In [18]:
# example query that the model will be based on
user_playlist_session_query = """
SELECT  
    artist,
    song,
    firstName,
    lastName
FROM user_playlist_session
WHERE userId = 10
    AND sessionId = 182
"""
                    

In [13]:
#create user_playlist_session table
user_playlist_session_table = """
CREATE TABLE IF NOT EXISTS user_playlist_session
    (
        userId        INT,
        sessionId     INT, 
        itemInSession INT,
        artist        TEXT, 
        firstName     TEXT, 
        lastName      TEXT,
        song          TEXT, 
        PRIMARY KEY   ((userId, sessionId), itemInSession)
    )
"""

try:
    session.execute(user_playlist_session_table)
except Exception as e:
    print(e)

In [14]:
# extract apporiate data from csv and load into user_playlist_session table
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    # skip header
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO user_playlist_session (userId, sessionId, itemInSession, artist, firstName, lastName, song)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        # rearrange order of csv values to match insert query 
        session.execute(query, ([int(line[10]), int(line[8]), int(line[3]), line[0], line[1], line[4], line[9]]))

### Validate table and SELECT query

In [19]:
#execute SELECT query
try:
    rows = session.execute(user_playlist_session_query)
except Exception as e:
    print(e)
    
#initialize output dataframe    
df_song_playlist_session = pd.DataFrame(columns=['artist', 'song', 'firstname', 'lastname'])

#append query output to dataframe
for row in rows:
    df_song_playlist_session = df_song_playlist_session.append(row, ignore_index=True)
df_song_playlist_session.head()

,artist,song,firstname,lastname
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


## Table 3: username

### Example Request
Get every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

### Primary Key
The `song` column is the partition keys because the associated `SELECT` query will filter results by `song`. The `userId` column is the first clustering column to prevent separate users with the same first and last name to be stored as one record. The artist column is a clustering column to prevent different artists with the same song to be stored as one record. 

In [20]:
# Example SELECT query
username_query = """
SELECT  firstName, 
        lastName
FROM username 
WHERE song = 'All Hands Against His Own'
"""
                    

In [21]:
# create username table
username_table = """
CREATE TABLE IF NOT EXISTS username 
    (
        song          TEXT,
        userId        INT,
        artist        TEXT, 
        firstName     TEXT, 
        lastName      TEXT,
        PRIMARY KEY   ((song), userId, artist, firstName, lastName)
    )
"""

try:
    session.execute(username_table)
except Exception as e:
    print(e)

In [22]:
# extract apporiate data from csv and load into username table
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    # skip header
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO username (song, userId, artist, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, ([line[9], int(line[10]), line[0], line[1], line[4]]))

### Validate table and select query

In [23]:
#execute SELECT query
try:
    rows = session.execute(username_query)
except Exception as e:
    print(e)
    
#initialize output dataframe
df_username = pd.DataFrame(columns=['firstname', 'lastname'])

# append query output to dataframe
for row in rows:
    df_username = df_username.append(row, ignore_index=True)
df_username.head()

,firstname,lastname
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


# Drop Tables

In [24]:
try:
    session.execute("DROP TABLE song")
except Exception as e:
    print(e)

try:
    session.execute("DROP TABLE user_playlist_session")
except Exception as e:
    print(e)

try:
    session.execute("DROP TABLE username")
except Exception as e:
    print(e)

# Close Session and Cluster Connection¶

In [3]:
session.shutdown()
cluster.shutdown()